# CCP-BioSim Training Course:
### Basic MD data analysis with MDTraj

IPython (Jupyter) notebooks are a great environment in which to do exploratory analysis of MD data. This notebook illustrates how two key Python packages - *numpy* and *matplotlib* - can be used with the MD analysis package [*mdtraj*](http://mdtraj.org/1.9.0/) for some simple standard analysis tasks.

I am assuming you have a basic knowledge of Python and Jupyter notebooks; if anything is unfamiliar a quick web search should enable you to fill in the gaps.

Along with this notebook you should have downloaded two data files:

1. **bpti.gro** : Structure file for the protein BPTI in Gromacs format.

2. **bpti.xtc** : Gromacs trajectory file for BPTI.

-----
To begin with, run the code in the following cell to check that you have all the data files and Python packages required:

In [ ]:
import os.path as op
problems = False
top_file = 'data/basic_analysis/bpti.gro'
traj_file = 'data/basic_analysis/bpti.xtc'
try:
    import mdtraj as mdt
except ImportError:
    print('Error: you don\'t seem to have mdtraj installed - use pip or similar to get it then try again.')
    problems = True
try:
    import numpy as np
except ImportError:
    print('Error: you don\'t seem to have numpy installed - use pip or similar to get it then try again.')
    problems = True
try:
    import matplotlib.pyplot as plt
    # This next line makes matplotlib show its graphs within the notebook.
    %matplotlib inline 
except ImportError:
    print('Error: you don\'t seem to have matplotlib installed - use pip or similar to get it then try again.')
    problems = True
try:
    import nglview as nv
except ImportError:
    print('Error: you don\'t seem to have nglview installed - use pip or similar to get it then try again.')
    problems = True

if not op.exists(top_file):
    print('Error: you don\'t seem to have the data file {} in this directory.'.format(top_file))
    problems = True
if not op.exists(traj_file):
    print('Error: you don\'t seem to have the data file {} in this directory.'.format(traj_file))
    problems = True

if problems:
    print('Fix the errors above then re-run this cell')
else:
    print('Success! You seem to have everything ready to run the notebook.')
plt.rcParams.update({'font.size': 15}) #This sets a better default label size for plots

## Preparation: loading the trajectory data.

Success? If so let's start by loading the MD data.

In the cell below, we create an MDTraj *Trajectory* object from the trajectory and topology files, then use the nglviewer app to have a quick look at the trajectory:

In [ ]:
traj = mdt.load(traj_file, top=top_file)
import nglview as nv
view = nv.show_mdtraj(traj)
view

Having got a feel for the molecule we are dealing with, and its dynamics, let's look at the Python representation of it.

Let's have a little dig around the trajectory object *traj*: 

In [ ]:
print(traj)
print(type(traj.xyz))
print(traj.xyz.shape)

So you can see that the trajectory coordinate data is stored in the numpy array *traj.xyz*. This is a three-dimensional array of shape [n_frames, n_atoms, 3]. 

**NB:** MDTraj stores coordinates in nanometer units.

---

## Example 1: Calculate and plot the end-to-end distance as a function of time.
Let's start with something simple. The cell below defines a function that calculates, for each frame, the distance between the first atom and the last. Then we apply the function to the trajectory object, and plot the result.

In [ ]:
def end_to_end(xyz):
    '''
    Calculate end to end distances
    
    Arguments:
        xyz : numpy array of shape (n_frames, n_atoms, 3)
    
    Returns:
        numpy array of shape (n_frames)
    '''
    d = np.zeros(len(xyz)) # create an array of zeros of length n_frames
    for i in range(len(xyz)):
        dxyz = xyz[i, 0] - xyz[i, -1] # note use of '-1' to get the last element of the array (the last atom)
        # The next two lines are just Pythagoras:
        dxyz2 = dxyz * dxyz
        d[i] = np.sqrt(dxyz2.sum()) # If you are not familiar with numpy sum() you might want to look it up
    return d

e2e = end_to_end(traj.xyz)
plt.plot(e2e)

### Follow-up challenges:

Tweak the code in the cell above to:

1. Add suitable x- and y-axis labels to the plot, and maybe a title (hint: the matplotlib documentation is [here](https://matplotlib.org)).
2. Plot the end-to-end distance in angstroms instead of nanometers.

---

## Example 2: Plotting RMSD data.
### 2a: The easy way.

*MDTraj* includes many useful trajectory analysis tools. Here we use the *rmsd* function to calculate the RMSD of each trajectory frame from the first, and then we plot it.

In [ ]:
rmsd = mdt.rmsd(traj, traj[0])
plt.plot(rmsd)

### 2b: The harder way.
Let's try using *numpy* to calculate the same thing. 

If you are quite new to *numpy* you may want to research what the *axis* argument to the *sum()* function does.

In [ ]:
def myrmsd(xyz):
    '''
    Calculate rmsd for a trajectory
    
    Arguments:
        xyz: numpy array of shape (n_frames, n_atoms, 3)
        
    Returns:
        numpy array of shape (n_frames)
    '''
    # calculate all displacements relative to the first frame:
    dxyz = xyz - xyz[0]
    # square them
    dxyz2 = dxyz * dxyz
    # Pythagoras: r2 = x2 + y2 + z2
    dr2 = dxyz2.sum(axis=2)
    # square root of mean value of r2 for each snapshot:
    rmsd = np.sqrt(dr2.mean(axis=1))
    return rmsd

rmsd2 = myrmsd(traj.xyz)
plt.plot(rmsd2) 

Whoops! That doesn't look right, does it? This is because it is standard practice when calculating RMSDs to least-squares fit the two snapshots together first, to remove translation and rotation. *MDTraj* did this, but the simple *numpy* code did not, so the RMSD values are much bigger. 

There is no simple way to fix this with *numpy*, but with *MDTraj* it is easy:

In [ ]:
traj.superpose(traj[0])

Now the trajectory data is all least-squares fitted to the first snapshot ("traj[0]"), we can try the simple *numpy* function again:

In [ ]:
rmsd3 = myrmsd(traj.xyz)
plt.plot(rmsd3)

### Follow-up challenge:
Well it looks like the *MDTraj* result, doesn't it? But to check, write some code in the cell below to calculate and plot the difference between the RMSDs calculated using the two approaches.

In [ ]:
# Write your code here:


---

## Example 3: Calculating RMSFs.
### 3a: The simple approach
While *MDTraj* has a simple function to calculate RMSDs, there is no equivalent for RMSFs (root-mean-square fluctuations) (Yes, there is an approach if you dig around a bit, but it's a bit clunky, IMHO). But with a bit of *numpy* its easy to create our own:

In [ ]:
def myrmsf(xyz):
    '''
    Calculate RMSF from trajectory data
    
    Arguments:
        xyz : numpy array of shape (n_frames, n_atoms, 3)
    
    Returns:
        numpy array of shape (n_atoms)
    '''
    # calculate all displacements relative to the mean structure:
    dxyz = xyz - xyz.mean(axis=0)
    # square them
    dxyz2 = dxyz * dxyz
    # Pythagoras: r2 = x2 + y2 + z2
    dr2 = dxyz2.sum(axis=2)
    # Square root of mean value of r2 for each atom, over all frames:
    rmsf = np.sqrt(dr2.mean(axis=0))
    return rmsf

rmsf = myrmsf(traj.xyz)
plt.plot(rmsf)

### Follow-up challenge:

Add suitable axis labels to the plot.

### 3b: A more advanced approach.
Instead of plotting the RMSF for each atom, how about the average RMSF for each residue? To do this we need to:

1. Find the index of the first atom in each residue
2. Average the atomic RMSFs from this atom to one less than the first atom of the next residue, or the last atom if it is the last residue.

To do the first you will see a bit of slightly more advanced wrangling with *MDTraj*, then the second is pretty straightforward:

In [ ]:
first_atoms = [r.atom(0).index for r in traj.topology.residues] # You may want to do a bit of research on this line.
def residue_rmsf(rmsf, first_atoms):
    '''
    Convert atom-resolution RMSFs into residue-resolution values
    
    Arguments:
        rmsf : rmsf data as numpy array of shape (n_atoms)
        first_atoms : indices of first atoms in each residue as numpy array of shape (n_residues)
        
    Returns:
        numpy array of rmsf values of shape (n_residues)
    '''
    n_res = len(first_atoms)
    result = np.zeros(n_res)
    for i in range(n_res - 1):
        result[i] = rmsf[first_atoms[i]:first_atoms[i+1]].mean()
    result[n_res - 1] = rmsf[first_atoms[n_res -1]:].mean()
    return result

res_rmsf = residue_rmsf(rmsf, first_atoms)
plt.plot(res_rmsf)

### Follow-up challenge:

You can get a numpy array with the indices of all the backbone (N, CA, C and O) atoms using the command:

```
backbone_list = traj.topology.select('backbone')
```

In the cell below write some code to calculate and plot the residue-based RMSF values for backbone atoms only:

In [ ]:
# Write your code here:


---

## Example 4: Secondary structure analysis.
*MDTraj* includes the DSSP algorithm to assign secondary structure. In its simplest form, this returns an [n_frames, n_residues] array where in each frame, each residue is labelled as 'H' (helical), 'E' (beta-strand), or 'C' (random coil):

In [ ]:
ss = mdt.compute_dssp(traj)
print (ss.shape)
print (ss[0])

Let's plot the number of helical residues in each snapshot. If you are fairly new to *numpy* then note that when the sum() function is applied to an array of logical (Boolean) values, it counts the number of True elements.

In [ ]:
n_helical = (ss == 'H').sum(axis=1)
plt.plot(n_helical)

In the cell below, we get a little cleverer and calculate the percentage native structure for each snapshot (assuming 'native' means 'same as the first snapshot'):

In [ ]:
plt.plot((ss == ss[0]).sum(axis=1) * 100 / 58)

### Follow-up challenge:
This graph might look nicer if the data was smoothed over, for example, a running window of 5 points. In the cell below create a function to do this and plot the results

In [ ]:
# Write your code here:


## Summary

Hopefully this brief introduction has convinced you that iPython/Jupyter notebooks are a very useful approach to MD data analysis. We have only scratched the surface of what *MDTraj* can do, and not even begun to look at how, with a bit of extra work, you can get publication-quality graphs out of *matplotlib*.